the purpose of this is  

In [1]:
import pandas as pd
import numpy as np
import pyarrow.parquet as pq

In [2]:
dataset_dir = 'child-mind-institute-problematic-internet-use/'
train_meta_file = 'train.csv'
test_meta_file = 'test.csv'
example_file = 'series_train.parquet/id=0a431608/part-0.parquet'

import data

In [3]:
train_df = pd.read_csv(dataset_dir + train_meta_file)
train_df

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii
0,00008ff9,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,...,4.0,2.0,4.0,55.0,NaN,NaN,NaN,Fall,3.0,2.0
1,000fd460,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,...,0.0,0.0,0.0,0.0,Fall,46.0,64.0,Summer,0.0,0.0
2,00105258,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,...,2.0,1.0,1.0,28.0,Fall,38.0,54.0,Summer,2.0,0.0
3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,...,3.0,4.0,1.0,44.0,Summer,31.0,45.0,Winter,0.0,1.0
4,0016bb22,Spring,18,1,Summer,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3955,ff8a2de4,Fall,13,0,Spring,60.0,Fall,16.362460,59.5,82.4,...,1.0,1.0,0.0,32.0,Winter,35.0,50.0,Fall,1.0,1.0
3956,ffa9794a,Winter,10,0,NaN,NaN,Spring,18.764678,53.5,76.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Winter,0.0,NaN
3957,ffcd4dbd,Fall,11,0,Spring,68.0,Winter,21.441500,60.0,109.8,...,1.0,0.0,1.0,31.0,Winter,56.0,77.0,Fall,0.0,1.0
3958,ffed1dd5,Spring,13,0,Spring,70.0,Winter,12.235895,70.7,87.0,...,1.0,1.0,1.0,19.0,Spring,33.0,47.0,Spring,1.0,0.0


In [4]:
# this finds the proportion of which there are no label

temp = train_df.isnull()
missing_portion = temp[temp['sii'] == True].sum()/len(temp[temp['sii'] == True])
missing_portion

id                                        0.000000
Basic_Demos-Enroll_Season                 0.000000
Basic_Demos-Age                           0.000000
Basic_Demos-Sex                           0.000000
CGAS-Season                               0.825980
                                            ...   
SDS-SDS_Total_Raw                         0.933007
SDS-SDS_Total_T                           0.933824
PreInt_EduHx-Season                       0.329248
PreInt_EduHx-computerinternet_hoursday    0.471405
sii                                       1.000000
Length: 82, dtype: float64

we want to predict the sii label. however, for those records who lack sii, a lot of other feature values are also missing.

we choose features that has this threshold that 